# Proteomic Data Aggregation and Visualization


This notebook demonstrates aggregation of proteomic data via the National Microbiome Data Collaborative (NMDC)'s [Runtime API](https://api.microbiomedata.org/docs). It highlights how the NMDC's schema can be used to overcome some of the numerous challenges associated with this type of aggregation. Please note that this notebook is intended for individuals with experience performing mass spectrometry based proteomic analyses and that various parameter and processing choices were made for this example use case. They are not broadly applicable and should be adjusted as needed. 


Notebook Steps:

1) Assess background information and collect datasets for an example study of riverbed sediment along the Columbia River

2) Apply a spectral probability filter across the data that optimizes the number of identifications for an FDR of 0.05

3) Collapse to unique peptides and normalize quantification

4) Extract functional gene annotations for proteins

5) Generate annotation and protein mappings for peptides using "Razor" strategy

6) Perform protein rollup using the "Razor" results and summarize into an aggregated table of relative protein abundance




Import libraries and python scripts containing functions necessary to run this notebook. 'aggregation_functions.py' (also in this folder) includes spectral probability filtering and protein mapping functions. 'nmdc_api.py' (located in NOM_visualization/python) includes functions for API traversal of the collections endpoint.

In [ ]:
# Setup 
# Add renv project library to R environment variable libPaths()
.libPaths(c(.libPaths(), "../../renv/library/*/R-*/*"))

# Load required packages
suppressPackageStartupMessages({
  library(dplyr, warn.conflicts = FALSE)
  library(tidyr, warn.conflicts = FALSE)
  library(stringr, warn.conflicts = FALSE)
  library(readr, warn.conflicts = FALSE)
  library(ggplot2, warn.conflicts = FALSE)
  library(jsonlite, warn.conflicts = FALSE)
  library(janitor, warn.conflicts = FALSE)
  library(grid, warn.conflicts = FALSE)
  })

# Load NMDC API functions from this repo
if(Sys.getenv("COLAB_BACKEND_VERSION") == "") {
  source("../../utility_functions.R")
  # import aggregation functions script
}

if(Sys.getenv("COLAB_BACKEND_VERSION") != "") {
  source("http://raw.githubusercontent.com/microbiomedata/nmdc_notebooks/refs/heads/main/utility_functions.R")
  # import aggregation functiosn script
}

## 1) Assess background information and collect data for an example study of riverbed sediment along the Columbia River

Review the example study on the [NMDC data portal](https://data.microbiomedata.org/details/study/nmdc:sty-11-aygzgv51). Use the study `id` embedded in the url (nmdc:sty-11-aygzgv51) to collect all related data objects via the [NMDC Runtime API](https://api.microbiomedata.org/docs) and reformat the json output into a pandas dataframe. These data objects reference both input files (i.e. raw, gff) and output files (i.e. metaproteomic results) to the NMDC workflows.

In [ ]:
data_objects <- get_data_objects_for_study("nmdc:sty-11-aygzgv51") %>%
  # Remove unnecessary columns for simpler dataframe
  select(id, name, file_size_bytes, data_object_type, md5_checksum, url, biosample_id, in_manifest) %>%
  # Flatten in_manifest
  mutate(in_manifest = as.character(in_manifest))

Subset the data objects to 'Unfiltered Metaproteomic Results'. These files contain the proteomic workflow outputs that will be used for proteomic aggregation.

In [ ]:
proteomic_output_df <- data_objects %>%
  filter(data_object_type == "Unfiltered Metaproteomics Results") %>%
  dplyr::rename(processed_dobj_id = "id")

head(proteomic_output_df)

There are various requirements that enable mass spectrometry runs to be aggregated and analyzed together. For example, runs need to be performed in succession, on the same instrument. The NMDC schema can make it easier to find these proteomic results by linking them via a slot called `in_manifest`.

Look at the `in_manifest` id on these proteomic outputs to confirm that all runs are in the same manifest record, and pull that record. If that manifest record's `manifest_category` value is 'instrument_run', then it confirms that these are LC-MS/MS runs that were performed in succession on the same instrument. Proteomic outputs from different manifest records should not be aggregated.

In [ ]:
# Display manifest IDs for the records in proteomic_output_df
manifest_id <- unique(proteomic_output_df$in_manifest)

# In this case there is only one, print manifest information
manifest <- get_results_by_id(collection = "manifest_set", 
                              match_id_field = "id", 
                              id_list = manifest_id, 
                              fields = "")
manifest

Look at an example of the information in 'Unfiltered Metaproteomics Results', which contains peptide identification and relative abundance information.

In [ ]:
paste("Reading file from", proteomic_output_df$url[1])

head(read_tsv(proteomic_output_df$url[1], show_col_types = FALSE, progress = FALSE))

Extract information from all 33 proteomic results via the function iterate_file_extract() in agg_func, and put them into a single dataframe, where each scan in each dataset has the unique identifier `SpecID`. Clean prefix and suffix off of each peptide sequence. Since this data was processed using a target-decoy approach, determine the type of protein being matched to each peptide: contaminant, reverse (false positive match to the reversed amino acid sequence of a protein), or forward (match to the true, forward amino acid sequence of a protein). The presence of forward and reverse matches enables FDR estimation in the next step.

In [2]:
# Define functions

gff_extract_features <- function(url) {
  
  withCallingHandlers(
    expr = {
      
      tsv_df <- suppressWarnings(read_tsv(url, col_names = FALSE, progress = FALSE, show_col_types = FALSE))
      
      colnames(tsv_df) <- c("seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute") 
      # See https://www.ensembl.org/info/website/upload/gff.html for GFF format specification
      
      # Break "attribute" column by separator
      tsv_df <- strsplit(tsv_df$attribute, split = ";") %>%
        lapply(strsplit, split = "=") %>%
        lapply(function (x) { 
          do.call(rbind, x) %>%
            t() %>%
            data.frame() %>%
            row_to_names(row_number = 1) }) %>%
        bind_rows() %>%
        distinct()
    },
    error = function(e) print(paste("Error while reading GFF from", url, ":", e))
  )
  return(tsv_df)
}



iterate_file_extract <- function(input_df, identifier_col, url_col, 
                                 extract_cols, file_type, 
                                 filter_col = NA, filter_values = NA) {
  
  output <- vector(mode = "list", length = nrow(input_df))

  for (row in 1:nrow(input_df)) {
    
    # Extract url and id for readability
    file_url <- input_df[[url_col]][row]
    identifier <- input_df[[identifier_col]][row]
    
    tryCatch(
      expr = {
        if(file_type == "tsv") {
          df <- read_tsv(file_url, show_col_types = FALSE, progress = FALSE)
          }
        if (file_type == "gff") {
          df <- gff_extract_features(file_url)
        }
      },
      error = function(e) print(paste("An error occurred fetching data from", identifier, ":", e))
    )
    
    # Check that the subsetted df will have unique rows, otherwise break
    if(nrow(df) != nrow(distinct(df[extract_cols]))) {
      print(paste("Selected columns result in non-unique rows for ", identifier, ". Data will not be included in output."))
      break
    }
    
    # Subset data frame to desired columns
    df <- df[extract_cols]
    
    # Filter if specified
    if (!is.na(filter_col) & all(is.na(filter_values))) {
      df <- filter(df, {{filter_col}} %in% filter_values)
    }
    
    # Add identifier column
    df <- mutate(df, id = identifier)
    
    # Append to list
    output[[row]] <- df
  }
  return(bind_rows(output))
}

trim_peptide_sequence <- function(s) {
  str_match(s, "\\.([A-Z\\\\*@#]+)\\.")[, 2]
}

In [ ]:
unfiltered_results <- iterate_file_extract(input_df = proteomic_output_df,
                                           identifier_col = "processed_dobj_id",
                                           url_col = "url", 
                                           extract_cols = c("Charge", "Scan", "Peptide", "Protein", "MSGFDB_SpecEValue", "StatMomentsArea"),
                                           file_type = "tsv") %>%


# #create identifier for each scan in each dataset
# unfilt_res["SpecID"] = unfilt_res.apply(lambda row: str(row["id_col"]) + "_" + str(row["Scan"]), axis=1)


  mutate(SpecID = paste(id, Scan, sep = "_")) %>%

# #clean off the prefix and suffix from the sequence but keep any mods
# unfilt_res["Peptide Sequence with Mods"] = unfilt_res["Peptide"].apply(agg_func.sequence_noprefsuff)
# del unfilt_res['Peptide']




  mutate(Peptide_Sequence_with_Mods = trim_peptide_sequence(Peptide)) %>%



# #determine protein type (contaminant, reverse, forward)
# unfilt_res["Protein_Type"] = unfilt_res["Protein"].apply(agg_func.findproteinname)

  mutate(Protein_Type = case_when(
    str_detect(Protein, "Contaminant") ~ "None",
    str_detect(Protein, "^XXX_") ~ "Reversed",
    TRUE ~ "Forward"))

# unfilt_res

head(unfiltered_results)

## 2) Apply a spectral probability filter across the data that optimizes the number of identifications for an FDR of 0.05

A challenge associated with aggregating mass spectrometry data is that there are always false identifications, which can be mitigated by imposing a spectral probability filter on the data being analyzed. The same spectral probability filter needs to be applied across datasets when they are being compared. The filter value itself is chosen by weighing the number of 'true' identifications retained with the proximity of the data to a chosen false discovery rate (FDR) (usually 0.05 or 0.01). NMDC's metaproteomic workflow provides 'true' and 'false' identifications for FDR estimation in the 'Unfiltered Metaproteomic Result' files.

Create a dataframe of peptide identifications (ignoring protein mapping). Filter identifications to the peptide sequence with the smallest SpecEValue for each SpecID, so there is a single, highest probability identification for each scan.

In [ ]:
edata <- distinct(unfiltered_results, SpecID, Peptide_Sequence_with_Mods, MSGFDB_SpecEValue, Protein_Type, StatMomentsArea, .keep_all = TRUE)



#for each SpecID, select the peptide spectrum match with the smallest MSGFDB_SpecEValue (.idxmin() takes the first entry when there's multiple matches)
# idx = edata.groupby(['SpecID'])['MSGFDB_SpecEValue'].idxmin()
# edata = edata.loc[idx].reset_index(drop=True)
# del idx

edata <- edata %>% 
  group_by(SpecID) %>% 
  slice_min(MSGFDB_SpecEValue, with_ties = FALSE, n = 1) %>% 
  ungroup()



# display(edata)

head(edata)

# assert len(edata['SpecID'].unique())==edata.shape[0], "still more than one identification per scan"

stopifnot("Still more than one identification per scan" = length(unique(edata$SpecID)) == length(edata$SpecID))

Create separate dataframes of forward and reverse peptide spectrum matches.

In [ ]:
forward_peptides <- filter(edata, Protein_Type == "Forward") %>% select(-Protein_Type)

head(forward_peptides)

In [ ]:
reversed_peptides <- filter(edata, Protein_Type == "Reversed") %>% select(-Protein_Type)

head(reversed_peptides)

Use the function optimize_specFilt() in agg_func to find a log10 spectral probability filter that weighs the number of forward peptides retained with the proximity of the dataset to a 0.05 spectral FDR. Visualize the impact of the spectral probability filter by plotting the number of forward and reverse peptides retained. 

The main plot below is a histogram of forward and reverse peptides across all spectral probability values. The inset within this plot depicts a subset of the smallest spectral probabililty values, with the red bar before the dashed line representing the estimated number of false identifications that will be included in this analysis. 

In [ ]:
#initial guess at a log10 spectral probability filter value
initial_specprob_filter = -15


spec_filt_value <- function(specprob, forward_peptides, reversed_peptides) {

  df_r <- filter(reversed_peptides, MSGFDB_SpecEValue < 10 ^ specprob)
  df_f <- filter(forward_peptides, MSGFDB_SpecEValue < 10 ^ specprob)

  f_spec <- length(unique(df_f$SpecID))
  r_spec <- length(unique(df_r$SpecID))
  
  fdr_spec <- ifelse(f_spec == 0 & r_spec == 0,
                     1,
                     (2 * r_spec) / (f_spec + r_spec))
    
  filter_value <- 1 / (0.050001 - fdr_spec) * (-f_spec)
  
  return(filter_value)
}





optimize_spec_filt <- function(initial_specprob_filter, forward_peptides, reversed_peptides) {

 result <- optim(fn = spec_filt_value, par = initial_specprob_filter, 
          forward_peptides = forward_peptides, reversed_peptides = reversed_peptides,
           method = "Brent", lower = -100, upper = 100)
 
 # Brent method for optim() function always returns 0 (success) for convergence
 
  tryCatch(
    expr = {
      result <- optim(fn = spec_filt_value, par = initial_specprob_filter, 
                      reversed_peptides = reversed_peptides, forward_peptides = forward_peptides,
                      method = "Brent", lower = -100, upper = 100)
      },
    error = function(e) message(paste("Error in optimization:", e)),
    warning = function(w) message(paste("Warning in optimization:", w))
  )
}

optimized_filter <- optimize_spec_filt(initial_specprob_filter, forward_peptides, reversed_peptides)$par


peps_for_plot <- bind_rows(forward_peptides, reversed_peptides, .id = "direction") %>%
  mutate(direction = case_when(direction == 1 ~ "forward", direction == 2 ~ "reverse"),
         direction = factor(direction))

main_plot <- ggplot(peps_for_plot) +
  geom_histogram(aes(x = MSGFDB_SpecEValue, fill = direction), bins = 50, alpha = 0.5, position = "identity") + 
  geom_vline(xintercept = 10 ^ optimized_filter) +
  scale_fill_manual(values = c("forward" = "seagreen", "reverse" = "orangered")) +
  ylab("Number of peptide-spectrum matches") +
  ggtitle("Impact of spectral probability filter")

zoom_plot <- peps_for_plot %>%
  # subset data - zoom in 
  filter(MSGFDB_SpecEValue < 2e-9) %>%
  ggplot() +
    geom_histogram(aes(x = MSGFDB_SpecEValue, fill = direction), bins = 30, alpha = 0.5, position = "identity") + 
    geom_vline(xintercept = 10 ^ optimized_filter) +
    scale_fill_manual(values = c("forward" = "seagreen", "reverse" = "orangered")) +
    theme(legend.position = "none", axis.title.x = element_blank(), axis.title.y = element_blank())
    
vp <- viewport(width = 0.4, height = 0.45, x = 0.6, y = 0.65)

main_plot
print(zoom_plot, vp = vp)





Apply the filter to the dataset and recalculate peptide and spectral FDR.

In [ ]:
forward_peptides <- filter(forward_peptides, MSGFDB_SpecEValue < 10 ^ optimized_filter)

# reversed_peptides = reversed_peptides[
#     (reversed_peptides["MSGFDB_SpecEValue"] < 10 ** optimization.x[0])
# ].copy()

reversed_peptides <- filter(reversed_peptides, MSGFDB_SpecEValue < 10 ^ optimized_filter)


# Calculate FDR
# f_spec = (forward_peptides["SpecID"].unique().size)
# r_spec = reversed_peptides["SpecID"].unique().size

f_spec <- length(unique(forward_peptides$SpecID))
r_spec <- length(unique(reversed_peptides$SpecID))


# if (f_spec == 0) & (r_spec == 0):
#     fdr_spec = 1
# else:
#     fdr_spec = (2*r_spec) / (f_spec + r_spec)

fdr_spec <- ifelse(f_spec == 0 & r_spec == 0,
                   1,
                   (2 * r_spec) / (f_spec + r_spec))

# f_pep = forward_peptides["Peptide Sequence with Mods"].unique().size
# r_pep = reversed_peptides["Peptide Sequence with Mods"].unique().size

f_pep <- length(unique(forward_peptides$Peptide_Sequence_with_Mods))
r_pep <- length(unique(reversed_peptides$Peptide_Sequence_with_Mods))


# if (f_pep == 0) & (r_pep == 0):
#     fdr_pep = 1
# else:
#     fdr_pep = (r_pep) / (f_pep + r_pep)


fdr_pep <- ifelse(f_pep == 0 & r_pep == 0,
                  1,
                  r_pep / (f_pep + r_pep))

# print("Spectral FDR:",fdr_spec,"\nPeptide FDR:",fdr_pep)


paste("Spectral FDR:", fdr_spec)
paste("Peptide FDR:", fdr_pep)


## 3) Collapse to unique peptides and normalize their relative abundance

At this point in analysis the data has been filtered to only high probability peptide identifications, but more than one scan within a dataset can have the same peptide identification. This can be due to the peptide eluting into the mass spectrometer over the course of multiple scans or a peptide eluting as multiple charge states. Sum the relative abundance for peptide sequences detected more than once in a dataset, leaving a total relative abundance value for each peptide in each dataset.

In [ ]:
forward_peptides <- forward_peptides %>%
  dplyr::rename(processed_dobj_id = id) %>%
  select(-c(SpecID, MSGFDB_SpecEValue)) %>%
  group_by(processed_dobj_id, Peptide_Sequence_with_Mods) %>%
  mutate(StatMomentsArea = sum(StatMomentsArea)) %>% 
  ungroup() %>%
  distinct(processed_dobj_id, Peptide_Sequence_with_Mods, StatMomentsArea)

head(forward_peptides)


Visualize the untransformed and un-normalized relative abundances.

In [ ]:
ggplot(forward_peptides) +
  geom_boxplot(aes(x = processed_dobj_id, y = StatMomentsArea)) +
  labs(x = "Samples", y = "Relative Peptide Abundance (Not Normalized)", title = "Peptide relative abundances by sample") +
  theme(axis.text.x = element_blank())

Apply log2 transformation and median normalize peptide abundances.

In [ ]:
forward_peptides <- forward_peptides %>%
  mutate(StatMomentsAreaLog2 = log2(StatMomentsArea)) %>%
  group_by(processed_dobj_id) %>%
  mutate(group_medians = median(StatMomentsAreaLog2)) %>%
  ungroup() %>%
  distinct()


# Calculate data wide median
# all_data_median=forward_peptides['StatMomentsAreaLog2'].median()

all_data_median <- median(forward_peptides$StatMomentsAreaLog2)

# Subtract sample wise median from each value within its group
# forward_peptides['StatMomentsAreaLog+Norm'] = forward_peptides.apply(
#     lambda row: row['StatMomentsAreaLog2'] - group_medians[row['processed_DO_id']], axis=1
# )

#add back in a data wide median value to avoid negative abundances
# forward_peptides['StatMomentsAreaLog+Norm'] = forward_peptides['StatMomentsAreaLog+Norm'] + all_data_median


forward_peptides <- forward_peptides %>%
  mutate(StatMomentsAreaLogNorm = StatMomentsAreaLog2 - group_medians + all_data_median)



# transformed_abundances_fig, ax = plt.subplots(figsize=(8,6))
# sns.boxplot(x='processed_DO_id',y='StatMomentsAreaLog+Norm',data=forward_peptides)
# plt.xticks([])
# plt.xlabel('sample')
# plt.ylabel('Relative Peptide Abundance (Normalized)')
# 
# del ax, group_medians, all_data_median, forward_peptides['StatMomentsArea'], forward_peptides['StatMomentsAreaLog2'], reversed_peptides


ggplot(forward_peptides) +
  geom_boxplot(aes(x = processed_dobj_id, y = StatMomentsAreaLogNorm)) +
  labs(x = "Samples", y = "Relative Peptide Abundance (Normalized)", title = "Peptide relative abundances by sample") +
  theme(axis.text.x = element_blank())

## 4) Extract functional gene annotations for proteins

Collect peptide to protein mapping information for the passing peptide sequences.

In [ ]:
peptide_protein_mapping <- unfiltered_results %>%
  filter(Peptide_Sequence_with_Mods %in% forward_peptides$Peptide_Sequence_with_Mods) %>%
  distinct(Peptide_Sequence_with_Mods, Protein)

peptide_protein_mapping

Annotation information for these proteins can be found in 'Functional Annotation GFF' files.

Since the `data_objects` dataframe contains all objects associated with our study id, it also contains the relevant 'Functional Annotation GFF' files. Subset this dataframe to GFF files associated with the 33 biosample ids that have a proteomic output in `proteomic_output_df`.

In [ ]:
annotation_input_df <- data_objects %>%
  filter(data_object_type == "Functional Annotation GFF") %>%
  filter(biosample_id %in% proteomic_output_df$biosample_id) %>%
  distinct(biosample_id, id, data_object_type, url)

# display(annotation_input_df)

head(annotation_input_df)

Preview the 'Functional Annotation GFF' files and determine a subset of gene annotation information that should be pulled from all 33 files.

In [ ]:
paste("Reading from", annotation_input_df$url[2])

head(gff_extract_features(annotation_input_df$url[2]))

Extract information from all 33 annotation files (this takes a while to run) and merge with `razormapping` so there is a final table of peptide-protein-annotation mapping (`annotation_mapping`).

In [ ]:
gene_mapping <- distinct(annotation_input_df, id, url)

# genemapping = agg_func.iterate_file_extract(
#     pd_df=genemapping,\
#     identifier_col='id',\
#     url_col='url',\
#     extract_cols=['ID','product','product_source'],\
#     filter_col = 'ID',
#     filter_values = peptide_protein_mapping['Protein'].unique().tolist(),
#     file_type='gff'
# )


gene_mapping <- iterate_file_extract(
  input_df = gene_mapping,
  identifier_col = "id",
  url_col = "url",
  extract_cols = c("ID", "product", "product_source"),
  filter_col = "ID",
  filter_values = unique(peptide_protein_mapping$Protein),
  file_type = "gff"
)


#merge with protein mapping information. drop columns ID (which is equivalent to Protein) and id_col (which is the dataset id, unnecessary here since peptide to protein information isn't dataset specific)
# annotation_mapping = genemapping.merge(peptide_protein_mapping,left_on='ID',right_on='Protein').drop(['ID','id_col'],axis=1)

annotation_mapping <- inner_join(gene_mapping, peptide_protein_mapping, by = join_by(ID == Protein)) %>%
  distinct()

annotation_mapping

## 5) Generate annotation and protein mappings for peptides using "Razor" strategy

Identify the razor protein, which is a method of limiting the assignment of degenerate peptides (i.e., peptides that map to more than one forward protein) to a most likely matched protein. 'Razor' references the principle Occam's razor, also known as the law of parsimony.

The rules are as follows:
- If a peptide is unique to a protein, then that protein is the razor
- Else, if a peptide belongs to more than one protein, but one of those proteins has a unique peptide, then that protein is the razor
- Else, if a peptide belongs to more than one protein and one of those proteins has the maximal number of peptides, then that protein is the razor
- Else, if a peptide belongs to more than one protein and more than one of those proteins has the maximal number of peptides, then collapse the proteins and gene annotations into single strings
- Else, if a peptide belongs to more than one protein and more than one of those proteins has a unique peptide, then the peptide is removed from analysis because its mapping is inconclusive

Use `annotation_mapping` as the input to the function razorprotein() from the agg_func script. This will return protein and gene annotation information for each peptide, according to the above rules. 

In [ ]:
# Add counts for use in razor logic function
# annotation_mapping has already been through distinct() so pairs (rows) are unique
annotation_mapping <- annotation_mapping %>%
  
  dplyr::rename(Protein = ID) %>%
  
  # Count the number of proteins that each peptide maps to
  group_by(Peptide_Sequence_with_Mods) %>%
  mutate(prot_count = n()) %>%
  ungroup() %>%
  
  # Count the number of REDUNDANT and UNIQUE peptides that each protein maps to
  group_by(Protein) %>%
  mutate(redundant_pep_count = sum(prot_count > 1),
         unique_pep_count = sum(prot_count == 1)) %>%
  ungroup()

annotation_mapping


get_razor_protein <- function(mapping_df) {
    
  # Make the mapping df into a bunch of named vectors for easier indexing
  
  # Names = peptides, values = number of proteins the peptide maps to
  a <- distinct(mapping_df, Peptide_Sequence_with_Mods, prot_count)
  prot_count_vec <- setNames(a$prot_count, a$Peptide_Sequence_with_Mods)
  
  # Names = peptides, values = proteins
  pep_prot_vec <- setNames(mapping_df$Protein, nm = mapping_df$Peptide_Sequence_with_Mods)
  
  # Create vector of all peptides for readability
  pep_vec <- a$Peptide_Sequence_with_Mods
  rm(a)
  

  # Pre allocate results list
  razor_result <- vector(mode = "list", length = length(pep_vec))
  
  # Iterate through peptides to choose a razor protein for each one
  for (pep in 1:length(pep_vec)) {
    
    query_peptide <- pep_vec[pep]
    
    # If the peptide only maps to one protein, that is the razor protein
    if (prot_count_vec[query_peptide] == 1) { razor_result[[pep]] <- pep_prot_vec[query_peptide] }
    
    # If the peptide maps to more than one protein and ...
    else {
      mapping_subset <- filter(mapping_df, Peptide_Sequence_with_Mods == query_peptide)
      prots_with_unique_peptides <- sum(mapping_subset$unique_pep_count > 0)
      
      razor_result[[pep]] <- case_when(
        
        # ...there is only one potential protein with unique peptides, that is the razor protein
        prots_with_unique_peptides == 1 ~ mapping_subset$Protein[which.max(mapping_subset$unique_pep_count)],
        
        # ...there is more than one potential protein with unique peptides, razor protein cannot be determined
        prots_with_unique_peptides > 1  ~ "indeterminate - discard",
        
        # ...there are no potential proteins with unique peptides, 
        # and ONE potential protein has the most redundant peptides, that is the razor protein
        # note: which.max returns the FIRST maximum index which in this case should be the only one
        prots_with_unique_peptides == 0 & 
          sum(mapping_subset$redundant_pep_count == max(mapping_subset$redundant_pep_count)) == 1 ~ 
          mapping_subset$Protein[which.max(mapping_subset$redundant_pep_count)],
        
        # ...there are no potential proteins with unique peptides, 
        # and more than one potential protein has the most redundant peptides, those are the razor proteins
        # note: which ( blah == max(blah) ) will return ALL of the maximum indices
        prots_with_unique_peptides == 0 &
          sum(mapping_subset$redundant_pep_count == max(mapping_subset$redundant_pep_count)) > 1 ~ 
          mapping_subset$Protein[which(mapping_subset$redundant_pep_count == max(mapping_subset$redundant_pep_count))],
        
        # there should be no cases not captured
        TRUE ~ "fix razor logic until you don't see this"
      )
    }
    razor_result[[pep]] <- data.frame(Peptide = query_peptide,
                                      Razor_Protein = razor_result[[pep]], 
                                      row.names = NULL)
  }
  # Bind into one long dataframe
  bind_rows(razor_result, .id = NULL) %>%
    
    # Discard indeterminate cases
    filter(Razor_Protein != "indeterminate - discard") %>%
    
    # Add the protein annotations back in
    left_join(select(mapping_df, Protein, product, product_source, Peptide_Sequence_with_Mods),
              by = join_by(Razor_Protein == Protein, Peptide == Peptide_Sequence_with_Mods)) %>%
    distinct()
}

razor_mapping_long <- get_razor_protein(annotation_mapping)

razor_mapping_oneline <- razor_mapping_long %>%
  group_by(Peptide) %>%
  mutate(Razor_Protein  = paste(Razor_Protein, collapse = ", "),
         product        = paste(product, collapse = ", "),
         product_source = paste(product_source, collapse = ", ")) %>%
  ungroup() %>%
  distinct()

razor_mapping_oneline


## 6) Perform protein rollup and summarize into a final aggregated table of relative protein abundance

Combine razor information with relative abundance values.

In [ ]:
forward_peptides <- forward_peptides %>%
  right_join(razor_mapping_oneline, by = join_by(Peptide_Sequence_with_Mods == Peptide)) %>%
  distinct()

forward_peptides

De-log the peptide abundances, sum the abundances for each razor protein and log transform the rolled up protein abundances.

In [ ]:

protein_abundances <- forward_peptides %>%
  mutate(StatMomentsAreaNorm = 2 ^ StatMomentsAreaLogNorm) %>%
  group_by(processed_dobj_id, Razor_Protein) %>%
  mutate(StatMomentsAreaNormSum = sum(StatMomentsAreaNorm)) %>%
  ungroup() %>%
  distinct(processed_dobj_id, product, product_source, Razor_Protein, StatMomentsAreaNormSum) %>%
  mutate(StatMomentsAreaLogNormSum = log2(StatMomentsAreaNormSum)) %>%
  select(-StatMomentsAreaNormSum)


protein_abundances

## Final aggregated table of relative protein abundance

Reformat these results into a proteomic table, where each row indicates a protein and each column indicates a sample/dataset. The values within are log transformed, median normalized relative abundance values. This table or the longform version above can be used in further proteomic analyses.

In [ ]:

aggregated_proteomic_output <- protein_abundances %>%
  select(processed_dobj_id, Razor_Protein, StatMomentsAreaLogNormSum) %>%
  pivot_wider(names_from = "processed_dobj_id", values_from = "StatMomentsAreaLogNormSum")


aggregated_proteomic_output

The generated protein table can be used as input to the software [pmartR](https://shinyproxy.emsl.pnnl.gov/app/pmart), which performs statistical analyses such as ANOVA and independence of missing data (IMD) tests. In this case, the aggregated proteomics table (`aggregated_proteomic_output`) would be equivalent to pmartR's `e_data` and the peptide to protein to gene mappings (`razor_mapping`) would be equivalent to pmartR's `e_meta`.

<img src="../pmartR_logo_final.jpg" width="25%"/>


pmartR requires sample metadata to parameterize analyses and interpret the data. For this example dataset, an API call will capture the biosample metadata (`sample_metadata`) that would be equivalent to pmartR's `f_data`.

Gather biosample metadata via the NMDC API `collection` endpoint, using the function get_id_results() in api_func and searching for the `biosample_id`s associated with each output in `proteomic_output_df`.

In [ ]:
biosample_metadata <- get_results_by_id(collection = "biosample_set",
                                        match_id_field = "id", 
                                        id_list = proteomic_output_df$biosample_id, 
                                        fields = "id,depth.has_numeric_value") %>%
  # Cleanup json output
  unnest(depth) %>%
  dplyr::rename(biosample_id = id,
                depth_m = has_numeric_value) %>%
  # Add data object IDs to connect biosample metadata to processed results
  left_join(select(proteomic_output_df, processed_dobj_id, biosample_id), by = join_by("biosample_id"))

biosample_metadata